In [1]:
using NearestNeighbors
using WordEmbeddings, SoftmaxClassifier
using Utils
using Query
using Base.Collections
using WordStreams

using Clustering
using Distances
using Iterators

In [2]:
ee = restore("models/ss/text8_200__i5.model");

In [23]:
function find_nearest_embedding2(embeddings_dtree, labels, target_embeddings::Matrix; nwords=5, banneds=[[]])
    max_words = nwords+maximum(map(length,banneds))                                                                 
    max_words = min(max_words,length(labels))                                                                       
    nwords = min(nwords,max_words)                                                                                  
    if length(banneds)==1                                                                                           
        banneds = repeated(banneds)                                                                                 
    end                                                                                                             
                                                                                                               
    idxss, distss = knn(embeddings_dtree,target_embeddings, max_words,true)                                         
    [Query._gather_nearests(nwords, idxs, dists, labels, banned) for (idxs, dists, banned) in zip(idxss,distss, banneds)] 
end 

find_nearest_embedding2 (generic function with 1 method)

In [30]:
function find_nearest_words2(embed::WordSenseEmbedding, word; nwords=5)                                                   wvs = hcat(embed.embedding[word]...)                                                                            
    dtree, labels = Query.nearest_neighbours_tree(embed)   
    wvs = hcat(embed.embedding[word]...)
    banneds=[[(word,si)] for si in 1:length(embed.embedding[word])]
    @show banneds
    find_nearest_embedding2(dtree, labels, wvs; nwords=nwords,  banneds=banneds)                                                     
                                                 
end                   

find_nearest_words2 (generic function with 1 method)

In [34]:
find_nearest_words2(ee,"dark")

In [35]:
ans

20-element Array{Array{Tuple{Tuple{String,Int64},Float32},1},1}:
 [(("wider",9),0.3889952f0),(("emits",1),0.391991f0),(("monopolization",1),0.39729744f0),(("financial",10),0.39756387f0),(("gifts",5),0.39910397f0)]     
 [(("aegean",20),0.38540563f0),(("history",15),0.3885529f0),(("history",12),0.39026806f0),(("late",11),0.39256102f0),(("ages",8),0.3933787f0)]           
 [(("presenting",12),0.39132434f0),(("muffed",1),0.39459732f0),(("rays",13),0.39779502f0),(("just",19),0.3994569f0),(("lego",8),0.4009274f0)]            
 [(("smack",1),0.39419347f0),(("hawke",3),0.40091893f0),(("linguists",19),0.40406385f0),(("repair",19),0.40457812f0),(("shortened",9),0.4054987f0)]      
 [(("coming",10),0.39665416f0),(("camelot",1),0.39722878f0),(("sino",16),0.39847133f0),(("cyrus",16),0.39953434f0),(("here",13),0.39985755f0)]           
 [(("plato",4),0.39481956f0),(("piano",8),0.40004236f0),(("lyric",1),0.40009555f0),(("concert",12),0.40305847f0),(("super",6),0.4043472f0)]              
 [(("matter

In [ ]:
fe=flatten_embeddings(ee)

In [ ]:
labels = Vector{typeof(fe[1][1])}(length(fe))
points = Vector{typeof(fe[1][2])}(length(fe))
@inbounds for (ii, (label, point)) in enumerate(fe)
    labels[ii]=label
    points[ii]=point
end
points = hcat(points...)
"";

In [ ]:
supertype(CosineDist)|>supertype|> supertype

In [ ]:
num_embeddings = sum(map(length,values(ee.embedding)))   

In [ ]:
BallTree |> supertype

In [ ]:
Matrix{Int}((4,5))

In [ ]:
subtypes(Metric)

In [ ]:
?RogersTanimoto

In [ ]:
"The metric equivelent of Cosine Distence"
immutable AngularDist<:Metric
end
function Distances.evaluate(::AngularDist, a::AbstractArray, b::AbstractArray)
    cdist = cosine_dist(a,b)
    cos_val = -cdist+1
    acos(cos_val)/π
    
end
angular_dist(a::AbstractArray, b::AbstractArray) = evaluate(AngularDist(), a, b)

In [ ]:
?clamp

In [ ]:
supertype(BallTree)

In [ ]:
@time dtree = BallTree(points, AngularDist())

In [ ]:
a= Float32[-0.39377338,0.00537273,-0.01265695,-0.10778123,-0.13810189,0.36825904,-0.109290004,-0.14393836,0.25121868,-0.27587888,0.27765274,-0.03303287,-0.377773,-0.035841785,-0.29538253,0.35813254,-0.10525085,0.056407243,0.38878596,0.063250735,0.035795324,0.08408793,-0.19964269,0.30433547,-0.14350525,0.10837218,-0.26265314,-0.07973809,-0.14241658,-0.04077801,0.07397601,-0.15602829,0.051554512,0.33332396,-0.11170346,0.2476235,-0.109009296,0.18476954,-0.070319,-0.18692827,-0.24492049,-0.09664763,0.066413775,0.24159898,-0.03155537,0.012303475,-0.2785114,0.49192646,0.014198316,-0.20214126,-0.24125631,0.3910152,-0.0403514,-0.1537596,-0.060242042,0.05657575,-0.12967439,0.004912391,-0.05168618,0.14485838,0.03324065,-0.32448116,0.027493626,-0.072312474,0.1430273,0.08515832,0.09191316,-0.01820727,-0.07709318,0.10794031,-0.3747032,0.09634692,0.0046729017,-0.07991098,-0.1842446,-0.10013454,0.17649256,0.07276382,-0.045141455,0.06433228,0.2105197,0.12094944,-0.0071323426,0.01847751,0.12829417,-0.12906034,-0.10412021,0.3214534,0.16969027,0.15265866,-0.2730719,0.3016221,-0.029113919,-0.38671944,0.028678758,-0.14230485,0.0051805535,0.14583129,-0.25782403,0.23371626,0.37521455,0.18324336,-0.06583255,-0.46758968,-0.256961,-0.16444625,0.004419465,0.0037241136,-0.06628683,0.047263525,-0.30958384,-0.1027391,0.030873004,-0.33358276,-0.27632016,-0.43650037,0.28573015,-0.064501,-0.04693959,0.36063057,0.3006676,0.20409773,0.010119039,0.046979215,0.021792123,0.14022377,0.11792495,0.17403485,-0.059884384,-0.089376114,-0.028302284,0.16705927,0.1545932,0.048059285,-0.16273244,0.20551379,-0.23519655,0.29807085,0.21797809,0.004707873,-0.028758567,0.18122742,-0.08651752,0.043573417,-0.20405923,0.18329029,-0.18415754,0.09676528,-0.18596773,0.1336391,0.22060071,-0.073048346,0.06798348,0.13525842,0.23386693,0.36470902,0.11507695,-0.15903345,-0.47879764,-0.14117014,0.3525215,-0.0109092295,-0.43467444,-0.108607784,-0.052173212,0.29667255,-0.19757205,-0.17868474,-0.3373953,-0.09163821,-0.3151417,0.0018271634,0.33228615,-0.07366353,-0.10422529,-0.005752259,0.07770148,-0.30420992,-0.20544204,-0.18041031,-0.30226153,0.06097357,-0.3809576,0.32778403,-0.12785572,-0.4702132,-0.510426,0.42915833,0.48551026,-0.25370514,0.18574406,-0.4405347,0.28528044,0.22997518,-0.17173867,0.027869938,-0.3262555,-0.11773469,0.117931806,-0.3186366]
b= Float32[-0.32254457,0.1303807,-0.011430562,0.20442386,-0.040092196,-0.20978203,0.12292641,0.13716105,-0.08172174,-0.018849451,-0.13544391,-0.12356674,0.18608235,0.17718397,-0.19190013,-0.06300311,-0.08640746,-0.029137397,-0.15641548,-0.35228166,0.15225682,-0.18051596,-0.11071779,0.514768,0.3428284,0.22844484,-0.11491289,-0.02501114,0.008015088,-0.08552168,-0.18874167,0.28050157,0.19153073,-0.1653949,-0.10701288,0.15500101,0.1894628,-0.17718585,-0.017673159,0.3100519,-0.016870338,-0.21239233,-0.1509686,-0.24261558,0.06907096,0.07523731,-0.07641739,0.0002398014,-0.25606146,0.1944692,-0.015224504,0.008161134,-0.053670477,-0.20095937,-0.42291957,-0.0131563125,-0.31735006,0.20073418,0.06847697,-0.030324949,0.122601725,-0.15902026,-0.06542995,0.11037707,-0.06387776,0.5150944,0.17905751,0.035313487,-0.28017563,0.16356237,0.28801802,-0.054847687,1.8513203e-5,0.088449195,-0.25639817,-0.04815431,0.3242258,-0.08836182,-0.21096031,-0.15345807,-0.19945858,0.06133386,0.109257124,0.10423821,0.09586634,-0.22308186,-0.14717077,0.022637988,-0.21134825,-0.06594093,-0.09484854,0.15860759,-0.24680427,0.28946087,0.046601977,0.10258316,-0.022382397,0.019213084,0.048615057,-0.033991523,-0.15809174,0.054869127,-0.020156877,0.054399412,0.11298344,-0.016594369,-0.019133871,0.3052367,-0.103136234,0.06930094,0.31171194,0.072627574,-0.3097729,-0.17790408,0.08344802,0.17666629,4.5090915e-6,0.2368096,0.31427222,0.109995745,0.03750653,0.038380373,-0.24993502,-0.09801941,0.2859209,0.21650718,0.28562385,0.026955936,-0.013438475,-0.09912299,-0.14297296,0.17945659,0.21791597,0.17862915,0.10613127,0.011857316,-0.20308164,-0.12614703,-0.09445148,0.054338038,-0.0726878,0.49233386,-0.19663678,0.21421023,0.030964443,0.070119105,0.041453857,0.05805054,0.16208911,-0.09511901,0.24685387,0.076273836,0.14853624,-0.051467616,-0.35248756,-0.18879889,0.17917107,-0.31694123,-0.0708936,0.025863899,-0.15934323,-0.0030867935,-0.07701962,0.17744702,0.083743595,-0.026958972,0.15422755,0.014264948,-0.07029209,-0.20464253,0.18610738,-0.034218706,-0.1035497,-0.031876855,0.27142885,0.033534583,0.22597368,-0.03695376,0.17813712,-0.34364215,0.42022872,0.3834817,-0.06578565,-0.13851804,-0.006017733,0.12881069,0.18563128,0.21414891,0.03130097,0.12905726,0.1307199,0.15027483,-0.027862698,-0.16566084,-0.25122535,0.00489496,-0.088786826,-0.24945556,-0.06253339,-0.093582764]
cosine_dist(a,b)

In [ ]:
acos(cosine_dist(ee.embedding["race"][2],ee.embedding["race"][10]))/π

In [ ]:
@time idxs, dists = knn(dtree, ee.embedding["rpg"][2], 6, true)
collect(zip(labels[idxs],dists))

In [ ]:
eltype([2])

In [ ]:
@time idxs, dists = knn(dtree, hcat(ee.embedding["rpg"]...), 6, trueid)
map(ids->labels[ids], idxs)

In [ ]:
[ee.embedding["race"][4] ee.embedding["race"][5]]

In [ ]:
@time Query.find_nearest_words(ee, "race", 5)

In [ ]:
ee.embedding["race"][2]